In [48]:
import random
import json
import pickle 
import numpy as np

import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import SGD, Adam 


Training the model

In [49]:
# Load the data
with open('intents.json') as file:
    data = json.load(file)
data

{'intents': [{'tag': 'Greetings',
   'patterns': ['Hi', 'Hello', 'Hey', "What's up"],
   'responses': ['Hey, how may I assist you today?',
    "What's up! Can I help you?"]},
  {'tag': 'Name',
   'patterns': ['What is your name?',
    'Who are you?',
    'what should I call you?',
    'Can you tell me your name?'],
   'responses': ['My name is GYMBRO. I am a chatbot here to assit you in your lifting journey.',
    'You can call me GYMBRO!',
    "I'm GYMBRO, your personal trainer!",
    " I'm GYMBRO! Nice to meet you."]},
  {'tag': 'Creator',
   'patterns': ['Who created you?',
    'Who is your maker?',
    'Who is your owner?'],
   'responses': ['GYMBRO was created by Asha Hopkins',
    'Asha Hopkins created me, GYMBRO, to assit mankind on their journey to self improvement through exercise!']},
  {'tag': 'Bye',
   'patterns': ['Thanks, see you later',
    'Bye',
    'Goodbye',
    'May the Gains be with you.'],
   'responses': ['Happy lifting!',
    'Go crush it in the gym',
    'Cheer

In [50]:
# Extract input and output data
inputs = []
outputs = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        inputs.append(pattern)
        outputs.append(intent['tag'])

In [51]:
# Tokenize the input data
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(inputs)
input_sequences = tokenizer.texts_to_sequences(inputs)
max_sequence_length = max(len(seq) for seq in input_sequences)
padded_input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_length)


In [52]:
# Create one-hot encoded output data
unique_tags = list(set(outputs))
output_indices = {tag:i for i, tag in enumerate(unique_tags)}
num_classes = len(unique_tags)
one_hot_outputs = tf.keras.utils.to_categorical([output_indices[tag] for tag in outputs], num_classes)

In [53]:
# Define the model architecture
inputs = Input(shape=(max_sequence_length,))
x = tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 128)(inputs)
x = LSTM(128)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

In [54]:
# Compile the model
optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_input_sequences, one_hot_outputs, epochs=200)

# Save the model
model.save('chatbot_model')

print('done')


Epoch 1/200
85/85 [==============================] - 7s 24ms/step - loss: 0.4837 - accuracy: 0.8729
Epoch 2/200
85/85 [==============================] - 2s 19ms/step - loss: 0.0543 - accuracy: 0.9779
Epoch 3/200
85/85 [==============================] - 2s 20ms/step - loss: 0.0451 - accuracy: 0.9805
Epoch 4/200
85/85 [==============================] - 2s 22ms/step - loss: 0.0420 - accuracy: 0.9812
Epoch 5/200
85/85 [==============================] - 2s 21ms/step - loss: 0.0391 - accuracy: 0.9838
Epoch 6/200
85/85 [==============================] - 2s 27ms/step - loss: 0.0373 - accuracy: 0.9853
Epoch 7/200
85/85 [==============================] - 2s 22ms/step - loss: 0.0369 - accuracy: 0.9853
Epoch 8/200
85/85 [==============================] - 2s 23ms/step - loss: 0.0368 - accuracy: 0.9853
Epoch 9/200
85/85 [==============================] - 2s 27ms/step - loss: 0.0372 - accuracy: 0.9853
Epoch 10/200
85/85 [==============================] - 2s 28ms/step - loss: 0.0345 - accuracy: 0.9853

INFO:tensorflow:Assets written to: chatbot_model/assets


INFO:tensorflow:Assets written to: chatbot_model/assets


done


Test Model

In [37]:
# Load the model
model = tf.keras.models.load_model('chatbot_model')


In [38]:
# Load the data
with open('intents.json') as file:
    data = json.load(file)

# Extract input and output data
inputs = []
outputs = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        inputs.append(pattern)
        outputs.append(intent['tag'])

In [39]:
# Tokenize the input data
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(inputs)

# Define a function to generate a response given a user input
def generate_response(model, tokenizer, max_sequence_length, input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, maxlen=max_sequence_length)
    output_probs = model.predict(padded_input_sequence)[0]
    predicted_output_index = tf.argmax(output_probs).numpy()
    predicted_output_tag = unique_tags[predicted_output_index]
    for intent in data['intents']:
        if intent['tag'] == predicted_output_tag:
            responses = intent['responses']
            return np.random.choice(responses)

In [40]:
# Test the model with some sample inputs
input_texts = ['Hi', 'I want a bigger back','I want to target my Abs', 'I want bigger quads', 'Exercise for biceps', 'Bye']
for input_text in input_texts:
    response = generate_response(model, tokenizer, max_sequence_length, input_text)
    print('Input: ', input_text)
    print('Response: ', response)

1/1 [==============================] - 1s 536ms/step
Input:  Hi
Response:  None
1/1 [==============================] - 0s 123ms/step
Input:  I want a bigger back
Response:  None
1/1 [==============================] - 0s 45ms/step
Input:  I want to target my Abs
Response:  None
1/1 [==============================] - 0s 49ms/step
Input:  I want bigger quads
Response:  None
1/1 [==============================] - 0s 56ms/step
Input:  Exercise for biceps
Response:  None
1/1 [==============================] - 0s 44ms/step
Input:  Bye
Response:  None


In [35]:
def liveChat():
    user_input = input("You: ") 
    while user_input != "bye":
        response = generate_response(model, tokenizer, max_sequence_length, user_input)
        print("You: " + user_input)
        print("GYMBRO: " + response)
        user_input = input("You: ") 

liveChat()

1/1 [==============================] - 0s 35ms/step
You: hi


TypeError: can only concatenate str (not "NoneType") to str